In [1]:
!pip install -q optuna


In [2]:
!pip install  -q monai


In [3]:
!pip install -q albumentations

In [4]:
!pip install -q git+https://github.com/huggingface/transformers.git

In [5]:
import os,sys
import numpy as np 
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from PIL import Image 
from tqdm.notebook import tqdm
from IPython.display import clear_output

In [6]:
#all required torch libraries
import torch
import torchvision
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms,utils
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torchvision.transforms import ToPILImage
from torch.utils.data.dataset import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.ops import masks_to_boxes
from albumentations import Compose, Transpose, VerticalFlip, HorizontalFlip, RandomRotate90, ShiftScaleRotate, OpticalDistortion

In [7]:

import time
import optuna
import csv
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("finished")
from os import listdir
from os.path import isfile, join

finished


In [8]:
import monai

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [9]:
def resize(path):
  dirs = os.listdir( path )
  for item in tqdm(dirs):
    if os.path.isfile(path+item):
      im = Image.open(path+item)
      f, e = os.path.splitext(path+item)
      imResize = im.resize((1024,1024), Image.NEAREST)
      imResize.save(f+e, 'PNG', quality=100)
      
label_path =  "/kaggle/input/nucleus-data-c-elegans/nucleus_data/segmentation_maps"
output_features_path = "/kaggle/input/nucleus-data-c-elegans/nucleus_data/features"
resize(label_path)

  0%|          | 0/6790 [00:00<?, ?it/s]

In [10]:
ids=[]
label_filenames = [f for f in listdir(label_path) if isfile(join(label_path, f))]
feature_filenames = [f for f in listdir(output_features_path) if isfile(join(output_features_path, f))]
for i in range(len(feature_filenames)):
  ids.append(feature_filenames[i][1:])
print(len(ids))

df = pd.DataFrame(ids ,columns=["file_ids"])
df.to_csv('full_file_ids.csv', index=False)

#sanity check
df = pd.read_csv('full_file_ids.csv')
df.head()

6756


,file_ids
0,182_22.png
1,167_27.png
2,86_29.png
3,154_16.png
4,177_8.png


In [11]:
import pandas as pd
import os
import cv2

df = pd.read_csv('full_file_ids.csv')
ids = df['file_ids'].tolist()
non_empty_ids = []

for file_id in ids:
    mask_path = os.path.join(label_path, 'L' + file_id)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if cv2.countNonZero(mask) > 0:
        non_empty_ids.append(file_id)

df_non_empty = pd.DataFrame(non_empty_ids, columns=["file_ids"])
df_non_empty.sort_values(by='file_ids', inplace=True)  # Sort the DataFrame by 'file_ids'
df_non_empty.to_csv('file_ids.csv', index=False)


dif = pd.read_csv('file_ids.csv')
dif.head(15)


,file_ids
0,0_10.png
1,0_11.png
2,0_12.png
3,0_13.png
4,0_14.png
5,0_15.png
6,0_16.png
7,0_17.png
8,0_18.png
9,0_19.png


In [12]:
pip install 'transformers[torch]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0
Note: you may need to restart the kernel to use updated packages.


In [13]:
class SegmentationDataset(Dataset):
    def __init__(self, csv, processor):
        self.df = pd.read_csv(csv)
        self.ids = self.df["file_ids"]
        self.processor = processor
        

    def __getitem__(self, idx):
        image = Image.open("/kaggle/input/nucleus-data-c-elegans/nucleus_data/features/F" + self.ids[idx])
        mask = np.array(Image.open("/kaggle/input/nucleus-data-c-elegans/nucleus_data/segmentation_maps/L" + self.ids[idx]))
           

        

        inputs = self.processor(image, return_tensors="pt")
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}

        inputs["ground_truth_mask"] = mask

        return inputs

    def __len__(self):
        return len(self.ids)
               

In [14]:
from transformers import SamProcessor

processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

In [15]:
train_dataset = SegmentationDataset(csv = "file_ids.csv", processor=processor)

In [16]:
example = train_dataset[0]
for k,v in example.items():
  print(k,v.shape)

pixel_values torch.Size([3, 1024, 1024])
original_sizes torch.Size([2])
reshaped_input_sizes torch.Size([2])
ground_truth_mask (256, 256)


In [29]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    

In [30]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
  print(k,v.shape)

pixel_values torch.Size([1, 3, 1024, 1024])
original_sizes torch.Size([1, 2])
reshaped_input_sizes torch.Size([1, 2])
ground_truth_mask torch.Size([1, 256, 256])


In [31]:
batch["ground_truth_mask"].shape


torch.Size([1, 256, 256])

In [32]:
from transformers import SamModel 

model = SamModel.from_pretrained("facebook/sam-vit-base")


In [33]:
for name, param in model.named_parameters():
  if name.startswith("vision_encoder") or name.startswith("prompt_encoder"):
    param.requires_grad_(False)

In [34]:
from torch.optim import Adam
import monai

# Note: Hyperparameter tuning could improve performance here
optimizer = Adam(model.mask_decoder.parameters(), lr=1e-5, weight_decay=0)

seg_loss = monai.losses.DiceCELoss(sigmoid=True, squared_pred=True, reduction='mean')


In [35]:
# !pip install torch-summary

In [36]:
# from torchsummary import summary
# summary(model,(3, 1024, 1024))

In [37]:
# !nvidia-smi

In [41]:
from tqdm import tqdm
from statistics import mean
import torch
from torch.nn.functional import threshold, normalize

num_epochs = 100

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.train()
for epoch in range(num_epochs):
    epoch_losses = []
    for batch in tqdm(train_dataloader):
        
        outputs = model(
            pixel_values=batch["pixel_values"].to(device),
            input_masks=batch["ground_truth_mask"].float().to(device),
            multimask_output=False,
        )

        # compute loss
        predicted_masks = outputs.pred_masks
        predicted_mask = torch.squeeze(predicted_masks, dim=1)
        ground_truth_masks = batch["ground_truth_mask"].float().to(device)
        loss = seg_loss(predicted_mask, ground_truth_masks.unsqueeze(dim=1))

        # backward pass (compute gradients of parameters w.r.t. loss)
        optimizer.zero_grad()
        loss.backward()

        # optimize
        optimizer.step()
        epoch_losses.append(loss.item())

    print(f"EPOCH: {epoch}")
    print(f"Mean loss: {mean(epoch_losses)}")


100%|██████████| 4978/4978 [27:49<00:00,  2.98it/s]


EPOCH: 0
Mean loss: 0.9956125541473786


100%|██████████| 4978/4978 [27:27<00:00,  3.02it/s]


EPOCH: 1
Mean loss: 0.9956059450722928


  2%|▏         | 118/4978 [00:39<26:47,  3.02it/s]


KeyboardInterrupt: 

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))